# Análise de Regressão - Olist Freight Value

## Objetivo
Prever o valor do frete (freight_value) usando diferentes modelos de regressão.

## Requisitos Obrigatórios
- ✅ Regressão Linear
- ✅ PCA para redução de dimensionalidade  
- ✅ 3 Métricas: R², MAE, RMSE

## Modelos Comparados
1. **Linear Regression** (obrigatório)
2. **Random Forest Regressor**
3. **Decision Tree Regressor**


In [1]:
# Importar bibliotecas necessárias
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

print("Bibliotecas importadas com sucesso!")


ModuleNotFoundError: No module named 'sklearn'

## 1. Carregamento dos Dados

In [ ]:
# Carregar datasets
order_items = pd.read_csv('data/olist_order_items_dataset.csv')
products = pd.read_csv('data/olist_products_dataset.csv')
orders = pd.read_csv('data/olist_orders_dataset.csv')
customers = pd.read_csv('data/olist_customers_dataset.csv')
sellers = pd.read_csv('data/olist_sellers_dataset.csv')

print("Dados carregados:")


## 2. Features

In [ ]:
# Criar feature de volume do produto
products['volume'] = (products['product_length_cm'] * 
                      products['product_height_cm'] * 
                      products['product_width_cm'])

# Preencher valores ausentes na categoria
products['product_category_name'] = products['product_category_name'].fillna('Indefinido')

print("Features criadas")

## 3. Merge e Preparação dos Dados

In [ ]:
# Merge dos dados
data = (order_items
        .merge(products, on='product_id')
        .merge(orders, on='order_id')
        .merge(customers, on='customer_id')
        .merge(sellers, on='seller_id'))

print("Dados unidos")


## 4. Criação de Features

In [ ]:
# Criar feature binária: mesmo estado?
data['same_state'] = (data['customer_state'] == data['seller_state']).astype(int)

# Codificar variáveis categóricas
data['customer_state'] = data['customer_state'].astype('category').cat.codes
data['seller_state'] = data['seller_state'].astype('category').cat.codes
data['product_category'] = data['product_category_name'].astype('category').cat.codes

# Definir features para o modelo
features = [
    'volume', 
    'product_weight_g', 
    'customer_state', 
    'seller_state', 
    'same_state', 
    'product_category'
]

# Preparar X e y (removendo NaN)
X = data[features].dropna()
y = data.loc[X.index, 'freight_value']

print(f"Features criadas: {features}")
print(f"Target (Frete) - média: R$ {y.mean():.2f}")


## 5. Train/Test Split e Normalização

In [ ]:
# Dividir em treino e teste (80/20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalizar os dados (necessário para PCA)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Divisão dos dados (20/80)")
print("Normalização aplicada com StandardScaler")

## 6. PCA - Redução de Dimensionalidade (Obrigatório)

In [ ]:
# Aplicar PCA mantendo 95% da variância
pca = PCA(n_components=0.95, random_state=42)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

print("PCA aplicado:")
print(f"Dimensões originais: {X_train_scaled.shape[1]}")
print(f"Dimensões após PCA: {X_train_pca.shape[1]}")
print(f"\nComponentes principais mantidos: {len(pca.explained_variance_ratio_)}")
print(f"Features originais: {features}")


## 7. Definição dos Modelos de Regressão

In [ ]:
# Definir os 3 modelos para comparação
models = {
    'Linear Regression': LinearRegression(),
    'Random Forest': RandomForestRegressor(
        n_estimators=100, 
        max_depth=15, 
        min_samples_leaf=2, 
        max_features='sqrt', 
        random_state=42, 
        n_jobs=-1
    ),
    'Decision Tree': DecisionTreeRegressor(
        max_depth=15, 
        min_samples_leaf=2, 
        ccp_alpha=0.0, 
        random_state=42
    )
}

print("Modelos definidos:")
for i, model_name in enumerate(models.keys(), 1):
    print(f"   {i}. {model_name}")


## 8. Treinamento e Avaliação dos Modelos

In [ ]:
# Dicionário para armazenar resultados
results = {}

# Treinar e avaliar cada modelo
for model_name, model in models.items():
    print(f"{'='*60}")
    print(f"Modelo: {model_name}")
    print(f"{'='*60}")
    
    # Treinar modelo
    model.fit(X_train_pca, y_train)
    
    # Predições
    y_pred = model.predict(X_test_pca)
    
    # Calcular as 3 métricas obrigatórias
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred) ** 0.5
    
    # Calcular acurácia com tolerância
    tolerance = 0.2  # 20% de tolerância
    accuracy = (abs(y_test - y_pred) / y_test <= tolerance).mean() * 100
    
    # Armazenar resultados
    results[model_name] = {
        'R²': r2,
        'MAE': mae,
        'RMSE': rmse,
        'Acurácia (±20%)': accuracy
    }
    
    # Exibir métricas
    print(f"R² Score: {r2:.4f}")
    print(f"MAE (Mean Absolute Error): R$ {mae:.2f}")
    print(f"RMSE (Root Mean Squared Error): R$ {rmse:.2f}")
    print(f"Predições dentro de ±20%: {accuracy:.2f}%")
    print()

print("Todos os modelos foram treinados e avaliados!")


## 9. Comparação dos Modelos

In [ ]:
# Tabela comparativa dos modelos
print(f"{'='*70}")
print("COMPARAÇÃO DOS MODELOS")
print(f"{'='*70}")
print(f"{'Modelo':<25} {'R²':<10} {'MAE':<12} {'RMSE':<12} {'Acurácia':<10}")
print(f"{'-'*70}")

for model_name, metrics in results.items():
    print(f"{model_name:<25} {metrics['R²']:<10.4f} R$ {metrics['MAE']:<9.2f} R$ {metrics['RMSE']:<9.2f} {metrics['Acurácia (±20%)']:<9.2f}%")

# Identificar o melhor modelo baseado em R²
best_model = max(results.items(), key=lambda x: x[1]['R²'])
print(f"\n{'='*70}")
print(f"Melhor modelo (por R²): {best_model[0]}")
print(f"   - R² Score: {best_model[1]['R²']:.4f}")
print(f"   - MAE: R$ {best_model[1]['MAE']:.2f}")
print(f"   - RMSE: R$ {best_model[1]['RMSE']:.2f}")
print(f"   - Acurácia (±20%): {best_model[1]['Acurácia (±20%)']:.2f}%")
print(f"{'='*70}")
